#1) Instalar librerias y conexión al servidor.

In [ ]:
# Instala Sodapy
!pip install sodapy

# Llamado a las librerías necesarias
import gspread
import numpy as np
import pandas as pd
import math
import requests
import itertools
import re
from datetime import datetime
from dateutil import parser
from google.colab import auth, files, data_table
from google.auth import default
from google.cloud import bigquery
from sodapy import Socrata
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from urllib.parse import urlparse, parse_qs

# Habilitar el formato para ver tablas en Google Colab
data_table.enable_dataframe_formatter()

# Autenticación de Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
client = gspread.authorize(creds)

#2) Descargar información & definir parametros.

In [ ]:
#1.
input_workbook_name = 'PRESUPUESTO 2025-2028F'
spreadsheet = gc.open(input_workbook_name)
sheet1 = spreadsheet.worksheet("CSF TOTAL")

data1_range = sheet1.get('B:C')
df = pd.DataFrame(data1_range[1:], columns=data1_range[0])
df.columns = ['ColumnaA','ColumnaB']

#2.
input_workbook_name_2 = 'Cambio Parametría MIS'
spreadsheet_2 = gc.open(input_workbook_name_2)
sheet2 = spreadsheet_2.worksheet("COME")

data2_range = sheet2.get('A:E')
df2 = pd.DataFrame(data2_range[1:], columns=data2_range[0])

#3.
input_workbook_name_3 = 'Parametria CSF'
spreadsheet_3 = gc.open(input_workbook_name_3)
sheet31 = spreadsheet_3.worksheet("Codigos_Neocon")
sheet32 = spreadsheet_3.worksheet("Codigos_Gestion")

data3_range = sheet31.get('A:D')
df3 = pd.DataFrame(data3_range[1:], columns=data3_range[0])

data4_range = sheet32.get('A:E')
df4 = pd.DataFrame(data4_range[1:], columns=data4_range[0])


#3) Parametria 1 - Nivel de Detalle: Pppto 2025-2028F.

*   Crear Niveles 0-n
*   Crear indicadores: Nivel Meximo, Nivel Anterior, Nivel, Ordenes cuenta maxima y cuenta individual, y Cantidad de Niveles.
*   Replica Niveles TD (Inverso)



In [ ]:
# Lista de nombres a eliminar
nombres_a_eliminar = ['GASTOS DE EXPLOTACION','Gastos de Administración ','ESTADO DE RESULTADOS','TOTAL MARGEN DE INTERESES','TOTAL MARGEN BRUTO','TOTAL MARGEN NETO','TOTAL RDOS. DE EXPLOTACIÓN ANTES DE IMPUESTOS','TOTAL RDOS. DE EXPLOTACIÓN NETOS']
df = df[~df['ColumnaA'].isin(nombres_a_eliminar)]

def cambiar_tipo(valor):
    if valor is None or pd.isna(valor) or valor == np.nan or valor == ""or valor=="0":
        return 'nulo'
    valor = str(valor).strip()  # Elimina espacios en blanco alrededor del valor y convierte a string
    if valor.isdigit():  # Verifica si el valor contiene solo números
        return 'numero'
    elif valor.isalpha():  # Verifica si el valor contiene solo letras
        return 'texto'
    else:
        return 'texto'  # Si es mixto o no es string, lo clasifica como 'texto'

df['tipo'] = df['ColumnaA'].apply(cambiar_tipo)

# Crear una copia del DataFrame original
df5 = df.copy()

df5['Nivel 1'] = df5.apply(lambda row: row['ColumnaB'] if row['tipo'] == 'nulo' else None, axis=1)

Niveles = max(len(list(g)) for k, g in itertools.groupby(df['tipo']) if k == 'texto')
nuevas_columnas = {f'Nivel {1 + i}': pd.NA for i in range(1, Niveles + 1)}

df_nuevas_columnas = pd.DataFrame(nuevas_columnas, index=df5.index)
df5 = pd.concat([df5, df_nuevas_columnas], axis=1)


# Inicializar las nuevas columnas con NA
for i in range(2, Niveles + 1):
    df5[f'Nivel {i}'] = pd.NA

# Inicialización de variables
in_text_block = False
current_level = 0  # Iniciar el Nivel en 0

# Iteración de abajo hacia arriba
for index, row in df5[::-1].iterrows():
    if row['tipo'] == 'texto':
        if not in_text_block:
            # Iniciar un nuevo bloque de texto y resetear el Nivel
            current_level = 0  # Restablecer el Nivel al inicio del bloque
            in_text_block = True

        # Incrementar el Nivel para la fila actual en el bloque de texto
        current_level += 1

        # Asignar el valor de 'ColumnaA' a la columna correspondiente
        df5.at[index, f'Nivel {current_level + 1}'] = row['ColumnaA']
    else:
        # Si encontramos un valor que no es 'texto', terminamos el bloque de texto
        in_text_block = False

Nivel_columns = [col for col in df5.columns if "Nivel" in col]
df5[Nivel_columns] = df5[Nivel_columns].fillna(method='ffill')


df5 = df5.fillna('')
df5 = df5.rename(columns={'ColumnaA': 'Cuenta Contable', 'ColumnaB': 'Nivel 0'})
columns_order = ['tipo'] + [col for col in df5.columns if col != 'tipo']
df5 = df5[columns_order]
df_resultado = df5[df5['tipo'] == 'numero']


#-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

df6 = df.copy()
columns_order = ['tipo'] + [col for col in df6.columns if col != 'tipo']
df6 = df6[columns_order]

df5 = df5.drop(columns=['Cuenta Contable'])

df6['ColumnaC'] = df6.apply(lambda row: row['ColumnaA'] if row['tipo'] == 'texto' else row['ColumnaB'], axis=1)

def find_level_general(value, df):
    filtered_df = df6[df6['ColumnaC'] == value]

    if filtered_df.empty:
        return 'No encontrado'

    if filtered_df['tipo'].values[0] == 'nulo':
        return 'Nivel 1'

    for col in df.columns[1:]:
        if value in df[col].values:
            return col

    return 'No encontrado'

df6['Nivel'] = df6['ColumnaC'].apply(lambda x: find_level_general(x, df5))
df6.loc[(df6['Nivel'] == 'Nivel 1') & (df6['tipo'] != 'nulo'), 'Nivel'] = 'Nivel 0'

next_col_mapping = {}
for col in df5.columns[:-1]:
    next_col_mapping[col] = df5.columns[df5.columns.get_loc(col) + 1]

def get_next_column_value(value, level_col, df5, next_col_mapping):
    if level_col in next_col_mapping:
        next_col = next_col_mapping[level_col]
        if value in df5[level_col].values:
            return df5[df5[level_col] == value][next_col].values[0]
    return ''

df6['Nivel Padre'] = df6.apply(
    lambda row: get_next_column_value(row['ColumnaC'], f'Nivel {row["Nivel"].split()[-1]}', df5, next_col_mapping),
    axis=1
)

df6.loc[df6['Nivel Padre'].isna() | (df6['Nivel Padre'] == ''), 'Nivel Padre'] = '(Nivel Maximo)'

df6['Nivel Maximo'] = df6.apply(
    lambda row: row['ColumnaC'] if row['Nivel Padre'] == '(Nivel Maximo)' else None,
    axis=1
)

df6.loc[df6['ColumnaC'] == 'Gastos de Personal', 'Nivel Maximo'] = df6['ColumnaC']
df6.loc[df6['ColumnaC'] == 'Contribuciones e Impuestos', 'Nivel Maximo'] = df6['ColumnaC']
df6.loc[df6['ColumnaC'] == 'Perdida por Deterioro de Activos', 'Nivel Maximo'] = df6['ColumnaC']
df6.loc[df6['ColumnaC'] == 'Dotacion a Provisiones', 'Nivel Maximo'] = df6['ColumnaC']
df6.loc[df6['ColumnaC'] == 'Impuesto Sociedades', 'Nivel Maximo'] = df6['ColumnaC']
df6['Nivel Maximo'] = df6['Nivel Maximo'].fillna(method='ffill')

df6['Orden Cuenta'] = range(1, len(df6) + 1)
df6['Orden Cuenta'] = df6['Orden Cuenta'].astype(int)

df6['Orden Cuenta - Nivel Maximo'] = df6.apply(
    lambda row: row['Orden Cuenta'] if row['Nivel Padre'] == '(Nivel Maximo)' else None,
    axis=1
)
df6['Orden Cuenta - Nivel Maximo'] = df6['Orden Cuenta - Nivel Maximo'].fillna(method='ffill')
df6['Orden Cuenta - Nivel Maximo'] = df6['Orden Cuenta - Nivel Maximo'].astype(int)

df6['Cantidad Niveles'] = df6.apply(
    lambda row: row['Nivel'] if row['Nivel Padre'] == '(Nivel Maximo)' else None,
    axis=1
)
df6['Cantidad Niveles'] = df6['Cantidad Niveles'].fillna(method='ffill')
df6['Cantidad Niveles'] = df6['Cantidad Niveles'].astype(str).str.replace('Nivel ', '', regex=False)
df6['Cantidad Niveles'] = df6['Cantidad Niveles'].astype(str).str.replace('No encontrado', '0', regex=False)
df6['Cantidad Niveles'] = df6['Cantidad Niveles'].astype(int)


df7 = df6.merge(df5, left_index=True, right_index=True)

nuevas_columnas_2 = {f'TD Nivel {1 + i}': pd.NA for i in range(-1, Niveles + 1)}
df_nuevas_columnas_2 = pd.DataFrame(nuevas_columnas_2, index=df5.index)
df7 = pd.concat([df7, df_nuevas_columnas_2], axis=1)


for i in range(Niveles+2):
    df7[f'TD Nivel {i}'] = df7[f'Nivel {Niveles+1 - i}']

def shift_left(row):
    values = [x for x in row if pd.notna(x) and x != ""]
    return pd.Series(values + [np.nan] * (len(row) - len(values)))
td_columns = [f'TD Nivel {i}' for i in range(Niveles+2)]
df7[td_columns] = df7[td_columns].apply(shift_left, axis=1)
df7 = df7.rename(columns={'ColumnaA': 'Cuenta Contable'})

df7.rename(columns=lambda x: x.replace('Nivel', 'BU Nivel') if x.startswith('Nivel') else x, inplace=True)
df7 = df7.rename(columns={'BU Nivel': 'Nivel','BU Nivel Padre':'Nivel Padre','BU Nivel Maximo':'Nivel Maximo'})
df7.rename(columns=lambda x: x.replace('_x', ''), inplace=True)
df7 = df7.dropna(subset=['ColumnaC'])
df7['Cuenta Contable'] = df7.apply(lambda row: row['Cuenta Contable'] if row['tipo'] == 'numero' else '', axis=1)
df7.drop(columns=['ColumnaB','tipo'], inplace=True)
df7 = df7.rename(columns={'ColumnaC': 'Nombre Cuenta Contable'})

<ipython-input-166-d7c132490617>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tipo'] = df['ColumnaA'].apply(cambiar_tipo)
<ipython-input-166-d7c132490617>:56: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df5[Nivel_columns] = df5[Nivel_columns].fillna(method='ffill')
<ipython-input-166-d7c132490617>:122: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df6['Nivel Maximo'] = df6['Nivel Maximo'].fillna(method='ffill')
<ipython-input-166-d7c132490617>:131: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or

#4) Cruces parametrias 2 y 3.


1.   Merged 1: Cruce Parametria 1 + Cambio Parametría MIS
2.   Merged 2: Merged 1 + Parametria CSF (Neocon)
3.   Merged 3: Merged 2 + Parametria CSF (Gestion).



In [ ]:
df_merged = pd.merge(df7, df2, how='left', left_on='Cuenta Contable', right_on='cuenta')

df_merged_2 = pd.merge(df_merged, df3, how='left', left_on='neocon', right_on='Cta. contable')

df_merged_2['gestion'] = pd.to_numeric(df_merged_2['gestion'], errors='coerce')
df4['C. GEST NUEV'] = pd.to_numeric(df4['C. GEST NUEV'], errors='coerce')
df4_sin_duplicados = df4.drop_duplicates(subset='C. GEST NUEV')
df_merged_3 = pd.merge(df_merged_2, df4_sin_duplicados, how='left', left_on='gestion', right_on='C. GEST NUEV')


df_merged_3 = df_merged_3.loc[:, ~df_merged_3.columns.str.contains('_y')]

df_merged_3 = df_merged_3.rename(columns={'': 'Detalle I'})
df_merged_3 = df_merged_3.rename(columns={'Detalle': ''})
df_merged_3 = df_merged_3.applymap(lambda x: x.strip() if isinstance(x, str) else x)

<ipython-input-167-6cafc0f24eb9>:15: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_merged_3 = df_merged_3.applymap(lambda x: x.strip() if isinstance(x, str) else x)


#5.1) Descargar información & definir parametros para Balance.


In [ ]:
# @title Texto de título predeterminado
input_workbook_name_4 = 'Consolidado Balances - Comercializadora'

spreadsheet4 = gc.open(input_workbook_name_4)
# Obtener datos de las hojas
sheet41 = spreadsheet4.worksheet("Balance_Resultado")
sheet42 = spreadsheet4.worksheet("Parametria_1")
sheet43 = spreadsheet4.worksheet("Inputs")
# Obtener fecha a actualizar
data5_range = sheet43.get('B1')
fecha_actualizar = data5_range[0][0]  # Asumiendo que 'C13' contiene la fecha como una lista de una celda
# Obtener fecha anteriror
data6_range = sheet43.get('B2')
fecha_anterior = data6_range[0][0]  # Asumiendo que 'C14' contiene la fecha como una lista de una celda

#ID de tu Google Sheet y nombre de la hoja
spreadsheet_id = '1t1EDsT9HQQkYL5QnsmV0u2wDagduXrFm4OHqjqak5-8'
sheet_name = 'Inputs'

# Abrir hoja de cálculo
sh = gc.open_by_key(spreadsheet_id)
worksheet = sh.worksheet(sheet_name)

sh = gc.open_by_key(spreadsheet_id)
worksheet = sh.worksheet(sheet_name)

# Definir rango donde se encuentran los enlaces (por ejemplo, desde C5 a C10)
start_row = 5
end_row = 100  # Cambia esto según la cantidad de enlaces que tengas

# Iterar sobre las celdas en la columna C
for row in range(start_row, end_row + 1):
    cell_value = worksheet.acell(f'B{row}').value  # Leer el enlace en la celda C{row}

    if cell_value:  # Verificar que la celda no esté vacía
        # Extraer el ID del archivo del enlace
        match = re.search(r'[-\w]{25,}', cell_value)
        if match:
            file_id = match.group(0)
            try:
                # Crear cliente de Google Drive
                drive_service = build('drive', 'v3', credentials=creds)
                file = drive_service.files().get(fileId=file_id).execute()
                file_name = file['name']
                worksheet.update_acell(f'C{row}', file_name)  # Escribir en D{row}

            except Exception:
                continue


def buscar_valor(matriz, fecha_actualizar):
    for fila in matriz:
        # Verificar que la fila tenga al menos tres elementos
        if len(fila) >= 3 and fila[0] == fecha_actualizar:  # fila[0] corresponde a la columna A
            return fila[2]  # fila[2] corresponde a la columna C
    return None  # Devolver None si no se encuentra el valor

# Suponiendo que la matriz está en el rango A4:C
matriz_range = sheet43.get('A4:C')
matriz = matriz_range

encabezado = matriz[0]
num_columnas = len(encabezado)
datos_ajustados = [fila + [np.nan] * (num_columnas - len(fila)) for fila in matriz[1:]]
data = pd.DataFrame(datos_ajustados, columns=encabezado)

def buscar_valor(matriz, fecha_actualizar):
    """
    Busca la fecha_actualizar en la columna A de la matriz y devuelve el valor correspondiente de la columna C.
    """
    for fila in matriz:
        # Verificar que la fila tenga al menos tres elementos
        if len(fila) >= 3 and fila[0] == fecha_actualizar:  # fila[0] corresponde a la columna A
            return fila[2]  # fila[2] corresponde a la columna C
    return None  # Devolver None si no se encuentra el valor

input_workbook_name_5 = buscar_valor(matriz, fecha_actualizar)
spreadsheet5 = gc.open(input_workbook_name_5)

# Obtener datos y construir DataFrames
data1_range = sheet41.get_all_values()
column_names = data1_range[0]
data = data1_range[1:]
df_temp = pd.DataFrame(data, columns=column_names)
columna_inicio = 'cuenta'
indice_inicio = df_temp.columns.get_loc(columna_inicio)
indice_final = df_temp.columns.get_loc(fecha_anterior)
df8 = df_temp.iloc[:, indice_inicio:indice_final + 1]

df9 = df7.copy()
df99 = df2.copy()

input_workbook_name_5 = buscar_valor(matriz, fecha_actualizar)
sheet5 = spreadsheet5.worksheet('Hoja1')
data4_range = sheet5.get('A:AB')
# Convertir el rango a un DataFrame
data4_df = pd.DataFrame(data4_range)
data4_df.iloc[13, 4] = "nombre_cuenta"
data4_df.columns = data4_df.iloc[13]
data4_df = data4_df[15:]
data4_df = data4_df.loc[:, data4_df.columns.notna() & (data4_df.columns != "")]
data4_df = data4_df.replace({None: np.nan})
df10 = data4_df.dropna(axis=1, how='all')
df10 = df10.rename(columns={'Account': 'cuenta 1','nombre_cuenta': 'nombre_cuenta 1'})
df9 = df9.rename(columns={'Cuenta Contable': 'cuenta 1','Nombre Cuenta Contable': 'nombre_cuenta 1'})

#5.2) Cruce bases.

Cruces:
*   Parameterias
*   Balance Primario: Historico / Balance Mensual
*   Balance Secundario: Balance Primario / Parametrias

In [ ]:
merged_df2 = pd.merge(df9, df8, left_on="cuenta 1", right_on="cuenta", how="right")
columnas_a_borrar = ['cuenta 1', 'nombre_cuenta 1']
merged_df2 = merged_df2.drop(columns=columnas_a_borrar, errors='ignore')

merged_df3 = pd.merge(merged_df2, df10[['cuenta 1', 'nombre_cuenta 1', 'Final Balance']],left_on="cuenta", right_on="cuenta 1", how="left")
merged_df3 = merged_df3.drop(columns=columnas_a_borrar, errors='ignore')
merged_df3 = merged_df3.rename(columns={'Final Balance': fecha_actualizar})

#5.3) Base de Nuevas Cuentas + Concantenado Balance

In [ ]:
# Nuevas cuentas
merged_df44 = pd.merge(df10[['cuenta 1', 'nombre_cuenta 1', 'Final Balance']], merged_df2[['cuenta']], left_on="cuenta 1", right_on="cuenta", how='left', indicator=True)
merged_df44 = merged_df44[merged_df44['_merge'] == 'left_only'].drop(columns=['_merge'])
merged_df44 = merged_df44.drop(columns='cuenta', errors='ignore')
merged_df44 = merged_df44.rename(columns={'cuenta 1': 'cuenta','nombre_cuenta 1': 'nombre_cuenta','Final Balance':'valor'})
merged_df44 = merged_df44[merged_df44['cuenta'] != ""]
merged_df44['cuenta'] = merged_df44['cuenta'].fillna('0')
merged_df44['cuenta'] = merged_df44['cuenta'].str.replace(',', '').astype(int)
merged_df44['valor'] = merged_df44['valor'].str.replace('.', '')
merged_df44['valor'] = merged_df44['valor'].str.replace(',', '.').astype(float)
merged_df44['Parameteria'] = merged_df44['cuenta'].apply(lambda x: any(str(x) in str(y) for y in df2['cuenta']))
merged_df44['Min_Lvl'] = merged_df44['cuenta'].apply(lambda x: not any(merged_df44['cuenta'].astype(str).str.startswith(str(x)) & (merged_df44['cuenta'] != x)))

merged_df4 = merged_df44.rename(columns={'valor': fecha_actualizar})
merged_df4 = merged_df4.rename(columns={'cuenta 1': 'cuenta','nombre_cuenta 1': 'nombre_cuenta'})

# Concatenar DataFrames
merged_df_concat = pd.concat([merged_df3, merged_df4], ignore_index=True)
merged_df_concat = merged_df_concat.drop(columns=['Parameteria', 'Min_Lvl'])


#5.4) Formato + Clasificaciones

1.   Formato de número
2.   Clasificacion tipo (B,P&L,O)
3.   Renombramiento columnas
4.   Reordenamiento columnas
6.   Creación filas "detalle"
7.   Clasificacion codigo (001,002,003)
7.   Clasificacion minimo lvl (TRUE,FALSE)
9.   Interpretación valores NaN

In [ ]:
def formato_solo_enteros(valor):
    try:
        # Reemplazar los puntos que son separadores de miles y las comas que son separadores decimales
        if isinstance(valor, str):
            valor = valor.replace('.', '').replace(',', '.')
        valor = int(float(valor))  # Convertir a float y luego a entero para manejar decimales
        return valor
    except (TypeError, ValueError):
        return valor  # Devolver el valor original si no se puede convertir

columns_to_fill = merged_df_concat.columns[merged_df_concat.columns.get_loc('nombre_cuenta') + 1:]

# Aplicar formato de solo enteros a las columnas seleccionadas antes de `fillna`
merged_df_concat[columns_to_fill] = merged_df_concat[columns_to_fill].applymap(formato_solo_enteros)

# Luego aplica `fillna`
merged_df_concat[columns_to_fill] = merged_df_concat[columns_to_fill].fillna(value=0)
#---------------------
#---------------------
# Obtener columnas de fechas
fechas = [col for col in merged_df_concat.columns if '/' in col]

# Convertir las columnas de fechas a numéricas
for fecha in fechas:
    merged_df_concat[fecha] = pd.to_numeric(merged_df_concat[fecha], errors='coerce')

# Filtrar cuentas que comienzan con 4 o 5
df_filtered = merged_df_concat[merged_df_concat['cuenta'].astype(str).str.startswith(('4', '5'))]

# Crear un DataFrame vacío para las nuevas filas
new_rows = []

for _, row in df_filtered.iterrows():
    cuenta = row['cuenta']
    nombre_cuenta = row['nombre_cuenta']

    detalle_name = f"{nombre_cuenta} + detalle"

    # Verificar si ya existe una fila con "+ detalle"
    existing_detalle_row = merged_df_concat[merged_df_concat['nombre_cuenta'] == detalle_name]

    if existing_detalle_row.empty:
        # Crear la fila con "+ detalle"
        new_row = {'cuenta': cuenta, 'nombre_cuenta': detalle_name}

        # Calcular las diferencias mes a mes
        prev_value = row[fechas[0]]
        differences = [0]  # La diferencia para el primer mes es 0

        for fecha in fechas[1:]:
            current_value = row[fecha]
            difference = current_value - prev_value
            differences.append(difference)
            prev_value = current_value

        # Añadir las diferencias al nuevo DataFrame
        new_row.update(dict(zip(fechas, differences)))
        new_rows.append(new_row)

    else:
        # Actualizar la fila existente con las nuevas diferencias
        existing_row_index = existing_detalle_row.index[0]
        prev_value = row[fechas[0]]

        updated_differences = [0]  # La diferencia para el primer mes es 0

        for fecha in fechas[1:]:
            current_value = row[fecha]
            difference = current_value - prev_value
            updated_differences.append(difference)
            prev_value = current_value

        merged_df_concat.loc[existing_row_index, fechas] = updated_differences

# Convertir las nuevas filas a un DataFrame y concatenar con el original
new_df = pd.DataFrame(new_rows)
merged_df_concat = pd.concat([merged_df_concat, new_df], ignore_index=True)

# Asegurarse de que no haya filas duplicadas para "+ detalle"
#merged_df_concat = merged_df_concat.drop_duplicates(subset=['cuenta',fecha_actualizar], keep='last')
# Eliminar filas con más de un "+ detalle"
def remove_extra_detalle(name):
    if name.count(' + detalle') > 1:
        return None
    return name

merged_df_concat['nombre_cuenta'] = merged_df_concat['nombre_cuenta'].apply(remove_extra_detalle)
merged_df_concat = merged_df_concat.dropna(subset=['nombre_cuenta'])
#---------------------------------------------------------------------------------------------------
merged_df_concat = merged_df_concat.replace('nan', '', regex=True)
merged_df_concat['cuenta'] = merged_df_concat['cuenta'].astype(str)

# Definir la función para categorizar cuentas
def categorizar_cuenta(cuenta):
    cuenta_str = str(cuenta) if pd.notna(cuenta) else ''
    if cuenta_str.startswith(('1', '2', '3')):
        return 'B'
    elif cuenta_str.startswith(('4', '5')):
        return 'P&L'
    else:
        return 'O'

# Aplicar categorización de cuentas
merged_df_concat['TIPO'] = merged_df_concat['cuenta'].apply(categorizar_cuenta)
columnas = ['TIPO'] + [col for col in merged_df_concat if col != 'TIPO']
merged_df_concat = merged_df_concat[columnas]

# Renombrar las columnas del DataFrame

original_columns = ['TIPO', 'LOCAL', 'LOCAL II', 'COD CONSOLIDACION', 'COD DE GESTION', 'NOMBRE CONSO', 'NOMBRE GESTION']
new_columns = ['TIPO', 'CM', 'CM Detalle', 'Neocon', 'Neocon Detalle', 'Gestión', 'Gestión Detalle']
column_mapping = dict(zip(original_columns, new_columns))
merged_df_concat = merged_df_concat.rename(columns=column_mapping)

# Obtener columnas a llenar
columns_to_fill = merged_df_concat.columns[merged_df_concat.columns.get_loc('nombre_cuenta') + 1:]
merged_df_concat[columns_to_fill] = merged_df_concat[columns_to_fill].fillna(value=0)

# Encuentra la posición de la columna 'nombre_cuenta'
nombre_cuenta_index = merged_df_concat.columns.get_loc('nombre_cuenta')
# Función para convertir el formato numérico
def convert_to_numeric(value):
    try:
        # Si es una cadena, reemplaza la coma decimal y convierte a float
        if isinstance(value, str):
            return float(value.replace('.', '').replace(',', '.'))
        return float(value)  # Para el caso en que ya sea un número
    except ValueError:
        return None  # Retorna None en caso de error de conversión

# Aplica la conversión a la columna especificada
merged_df_concat[fecha_actualizar] = merged_df_concat[fecha_actualizar].apply(convert_to_numeric)


merged_df_concat_new=merged_df_concat
#---------------------------------------------------------------------------------------------------------------------------------
# Definir la función para asignar valores a la nueva columna
def assign_code(row):
    nombre_cuenta_str = str(row['nombre_cuenta'])
    cuenta_str = str(row['cuenta'])
    if 'detalle' in nombre_cuenta_str:
        return '003'
    elif cuenta_str.startswith(('1', '2', '3')):
        return '001'
    elif cuenta_str.startswith(('4', '5')):
        return '002'
    else:
        return ''

# Crear la nueva columna con valores basados en la condición
merged_df_concat_new['Clasificacion'] = merged_df_concat_new.apply(assign_code, axis=1)

merged_df_concat_new['Min_Lvl'] = merged_df_concat['cuenta'].apply(lambda x: not any(merged_df_concat['cuenta'].str.startswith(x)& (merged_df_concat['cuenta']!=x)))

# Reordenar las columnas para colocar 'new_column' antes de 'cuenta'
columns = merged_df_concat_new.columns.tolist()
cuenta_index = columns.index('cuenta')

# Insertar 'new_column' antes de 'cuenta'
columns.insert(cuenta_index, columns.pop(columns.index('Clasificacion')))
merged_df_concat_new = merged_df_concat_new[columns]

# Reordenar las columnas para colocar 'Min_Lvl' antes de 'cuenta'
columns = merged_df_concat_new.columns.tolist()
cuenta_index = columns.index('cuenta')

# Insertar 'Min_Lvl' antes de 'cuenta'
columns.insert(cuenta_index, columns.pop(columns.index('Min_Lvl')))
merged_df_concat_new = merged_df_concat_new[columns]

merged_df_concat_new[columns_to_fill] = merged_df_concat_new[columns_to_fill].fillna(value=0)
merged_df_concat_new[columns_to_fill] = merged_df_concat_new[columns_to_fill].applymap(formato_solo_enteros)

merged_df_concat_new = merged_df_concat_new.sort_values(by='cuenta', ascending=True)

merged_df_concat_new['cuenta'] = merged_df_concat_new['cuenta'].str.replace(',', '').astype(float)

# Si deseas restablecer los índices después de ordenar
merged_df_concat_new.reset_index(drop=True, inplace=True)
merged_df_concat_new = merged_df_concat_new[merged_df_concat_new['cuenta'].notna() & (merged_df_concat_new['cuenta'] != 0)]

<ipython-input-171-1f03a7e0f734>:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df_concat[columns_to_fill] = merged_df_concat[columns_to_fill].applymap(formato_solo_enteros)
<ipython-input-171-1f03a7e0f734>:172: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  merged_df_concat_new[columns_to_fill] = merged_df_concat_new[columns_to_fill].applymap(formato_solo_enteros)


#6) Actualizar hojas de cálculo en Google Sheets

In [ ]:
df_merged_3['Cuenta Contable'] = df_merged_3['Cuenta Contable'].fillna("0")
df_merged_3['Cuenta Contable'] = df_merged_3['Cuenta Contable'].replace("", "0")
df_merged_3['Cuenta Contable'] = df_merged_3['Cuenta Contable'].str.replace(',', '').astype(int)
df_merged_3['cuenta'] = df_merged_3['cuenta'].fillna("0")
df_merged_3['cuenta'] = df_merged_3['cuenta'].str.replace(',', '').astype(int)
df_merged_3['neocon'] = df_merged_3['neocon'].fillna("0")
df_merged_3['neocon'] = df_merged_3['neocon'].str.replace(',', '').astype(int)
df_merged_3['Cta. contable'] = df_merged_3['Cta. contable'].fillna("0")
df_merged_3['Cta. contable'] = df_merged_3['Cta. contable'].str.replace(',', '').astype(int)
output_sheet_balance_resultado = gc.open('Consolidado Balances - Comercializadora').worksheet('Parametria_1')
output_sheet_balance_resultado.clear()
output_sheet_balance_resultado.update([df_merged_3.columns.values.tolist()] + df_merged_3.fillna("").values.tolist())

df2['cuenta'] = df2['cuenta'].str.replace(',', '').astype(int)
df2['gestion'] = df2['gestion'].fillna("0")
df2['gestion'] = df2['gestion'].str.replace(',', '').astype(int)
df2['neocon'] = df2['neocon'].fillna("0")
df2['neocon'] = df2['neocon'].str.replace(',', '').astype(int)
output_sheet_balance_resultado = gc.open('Consolidado Balances - Comercializadora').worksheet('Parametria_2')
output_sheet_balance_resultado.clear()
output_sheet_balance_resultado.update([df2.columns.values.tolist()] + df2.fillna("").values.tolist())

# Actualizar hojas de cálculo en Google Sheets
output_sheet_balance_resultado_2 = gc.open(input_workbook_name_4).worksheet('Balance_Resultado')
output_sheet_balance_resultado_2.clear()
output_sheet_balance_resultado_2.update([merged_df_concat_new.columns.values.tolist()] + merged_df_concat_new.fillna('').values.tolist())

output_sheet_balance_resultado_3 = gc.open(input_workbook_name_4).worksheet('Nuevas_Cuentas')
existing_data = output_sheet_balance_resultado_3.get_all_values()
if 'fecha_actualizar' not in merged_df44.columns:
    merged_df44['fecha_actualizar'] = fecha_actualizar
header = merged_df44.columns.tolist()
new_data = merged_df44.fillna('').values.tolist()
if existing_data:
    combined_data = existing_data[1:] + new_data
else:
    combined_data = new_data
output_sheet_balance_resultado_3.update([header] + combined_data)

sheet66 = spreadsheet4.worksheet("Nuevas_Cuentas")
data6_range = sheet66.get('A:F')
df66 = pd.DataFrame(data6_range[1:], columns=data6_range[0])
df66['cuenta'] = df66['cuenta'].str.replace(',', '').astype(int)
df66['valor'] = df66['valor'].str.replace(',', '').astype(int)
df66['validacion'] = df66['cuenta'].isin(merged_df_concat_new['cuenta'])
df66 = df66.sort_values(by='fecha_actualizar')
df66 = df66.drop_duplicates(subset='cuenta', keep='first')
df66 = df66[df66['Parameteria'] != 'TRUE']

output_sheet_balance_resultado = gc.open(input_workbook_name_4).worksheet('Nuevas_Cuentas')
output_sheet_balance_resultado.clear()
output_sheet_balance_resultado.update([df66.columns.values.tolist()] + df66.fillna('').values.tolist())